# Twitter Project

 -  we will create a simple application that plots out the popularite of tags associated with incoming tweets streamed live from twiter
 
 - first we need to create a Twitter developer Account to get out acces codes 
 
 - Install tweepy lirary as we well Matplotlib ans Seaborn
 
 
 

# So out next Step is to write a script to connect to our Twitter Streaming 

- the file is Twitter Connection

In [1]:
import tweepy
import tweepy
from tweepy import OAuthHandler, Stream
from tweepy.streaming import Stream
import socket, json
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark import SparkContext
from pyspark.streaming import StreamingContext



In [2]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

# Create Spark context 

In [ ]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [ ]:
socket_stream = ssc.socketTextStream(("your_ip_adress", "the port ")) # (255.255.255.255, 8888)

In [ ]:
lines = socket_stream.window( 20 )

In [ ]:
from collections import namedtuple # quick way to create class or tuple 
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [ ]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) ) # Registers to a table.



# all that what it means is for every ten twets that we read in, we are going to create state of frame wich has 
#(tag, couunt) and that dataframe is going to be registred as a temporary table called tweets later on.

# Now Run Twitter connextion.py

In [ ]:
ssc.start()

In [ ]:
# display all this mechanism
import time
from IPython import display # allows show stuff directely in jupyter 
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [ ]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True) # refresh out plot 
    sns.plt.figure( figsize = ( 10, 8 ) ) # create the figure
    sns.barplot( x="count", y="tag", data=top_10_df)
    sns.plt.show()
    count = count + 1

In [ ]:
ssc.stop()